In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

In [3]:
def build_generator(hidden1, hidden2, hidden3):
    model = Sequential()
    model.add(Dense(hidden1, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)

In [4]:
def trainGAN(gen_hidden1, gen_hidden2, gen_hidden3):
    batch_size = 256
    epochs = 7000
    optimizer = Adam(0.0002, 0.5)
    
    dataframe = pd.read_csv('satan.csv').sample(9000) # sample 100 data points randomly from the csv
    
    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    #to visually judge results
    print("Real satan attacks:")
    print(dataset[:2])
    
    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # build the discriminator portion
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # build the generator portion
    generator = build_generator(gen_hidden1, gen_hidden2, gen_hidden3)
    
    #input and output of our combined model
    z = Input(shape=(41,))
    attack = generator(z)
    validity = discriminator(attack)
    
    # build combined model from generator and discriminator
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #break condition for training (when diverging)
    loss_increase_count = 0;
    prev_g_loss = 0;
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]
        
        # generate a matrix of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 41))
        
        # create an array of generated attacks
        gen_attacks = generator.predict(noise)
        
        # loss functions, based on what metrics we specify at model compile time
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # generator loss function
        g_loss = combined.train_on_batch(noise, valid)
        
        if epoch % 100 == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss change: %.3f, Loss increases: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        prev_g_loss = g_loss
            
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
            
        if epoch % 20 == 0:
            f = open("GANresultsSatan.txt", "a")
            np.savetxt("GANresultsSatan.txt", gen_attacks, fmt="%.0f")
            f.close()
            
    # peek at our results
    results = np.loadtxt("GANresultsSatan.txt")
    print("Generated Satan attacks: ")
    print(results[:2])
        
        


In [5]:
dataframe = pd.read_csv("kdd_dataset.csv")
dataframe = dataframe.sample(n=3000)

le = LabelEncoder()
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
num_of_classes = len(attack_labels)
dataset = dataframe_encoded.values

indices_of_satan = np.where(attack_labels == 'satan.')
satan_index = indices_of_satan[0]

X = dataset[0:3000,0:41].astype(float)
Y = dataset[0:3000,41]


In [6]:
def baseline_model():
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 18
    hidden_layer2 = 6
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=300, batch_size=10, verbose=0)

estimator.fit(X, Y)
y_pred = estimator.predict(X)

predicted_as_label = attack_labels[y_pred]
unique_labels = np.unique(predicted_as_label)

for label in unique_labels:
    print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))

cm = confusion_matrix(Y, y_pred)
print(cm)
print("accuracy: " + str(np.trace(cm) / cm.sum()))

Attack type: back.     number predicted:  13
Attack type: ipsweep.     number predicted:  6
Attack type: land.     number predicted:  1
Attack type: neptune.     number predicted:  626
Attack type: nmap.     number predicted:  1
Attack type: normal.     number predicted:  616
Attack type: pod.     number predicted:  2
Attack type: portsweep.     number predicted:  6
Attack type: satan.     number predicted:  19
Attack type: smurf.     number predicted:  1702
Attack type: teardrop.     number predicted:  2
Attack type: warezclient.     number predicted:  6
[[  12    0    0    0    0    0    0    0    0    0    0    0]
 [   0    6    0    0    0    0    0    0    0    0    0    0]
 [   0    0    1    0    0    0    0    0    0    0    0    0]
 [   0    0    0  626    0    0    0    0    0    0    0    0]
 [   0    0    0    0    1    0    0    0    0    0    0    0]
 [   1    0    0    0    0  616    0    0    0    0    0    0]
 [   0    0    0    0    0    0    2    0    0    0    0    

In [8]:
"""
f = open("GeneratorHypersAbove50percentAccuracy.txt", "w")
f.write("""""" Hidden layer counts for Generator model that resulted in over 50% generated attacks labeled correctly:
    ------------------------------------------------------------------------------------------------
    """""")
f.close()
"""

while(1):
    # generate random numbers for the hidden layer sizes of our generator
    gen_hidden1 = np.random.randint(1, 101)
    gen_hidden2 = np.random.randint(1, 101)
    gen_hidden3 = np.random.randint(1, 101)
    
    i = 0
    
    
    # train 5 times on each setup, in case we get unlucky initalization on an otherwise good setup
    while i < 5:
        # create a unique filename in case we want to store the results (good accuracy)
        result_filename = "GANresultsSatan%.0f%.0f%.0fiter%.0f.txt" % (gen_hidden1, gen_hidden2, gen_hidden3, i)

        trainGAN(gen_hidden1, gen_hidden2, gen_hidden3)
        
        # load generate attacks from file
        results = np.loadtxt("GANresultsSatan.txt")

        # predict attack lables (as encoded integers)
        y_pred = estimator.predict(results)

        # create appropriate labels for our generated neptune attacks
        satan_labels = np.full((len(results),), satan_index[0])

        # convert integer labels back to string, get all unique strings and their count
        predicted_as_label = attack_labels[y_pred]
        unique_labels = np.unique(predicted_as_label)

        for label in unique_labels:
            print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))
    
        print()
        # create a confusion matrix of the results
        cm = confusion_matrix(satan_labels, y_pred)
        
        accuracy = np.trace(cm) / cm.sum()
        print(cm)
        print("total: " + str(cm.sum()))
        print("accuracy: " + str(accuracy))
        
        if accuracy > .50:
            f = open("GeneratorHypersAbove50percentAccuracySatan.txt", "a")
            f.write("""
            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: %.3f
Generator hidden layer 1 size: %.0f
Generator hidden layer 2 size: %.0f
Generator hidden layer 3 size: %.0f
Iteration %.0f
Result file name: %s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~""" % (accuracy, gen_hidden1, gen_hidden2, gen_hidden3, i, result_filename))
            f.close()
            
            f = open(result_filename, "w")
            f.close()
            np.savetxt(result_filename, results, fmt="%.0f")
        
        i = i + 1
            


Real satan attacks:
[[  0   1  40   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 504   0   0   1   1   0   0   1   0 174   0   0  99   0
    0  42   1  58   0   0]
 [  0   1  40   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 501   0   0   0   1   1   0   1   0 174   0   0  99   0
    0   7   0  93   7   0]]
0 [D loss: 0.364126, acc.: 62.89%] [G loss: 0.595235] [Loss change: 0.595, Loss increases: 0]
100 [D loss: 0.662544, acc.: 50.78%] [G loss: 0.387241] [Loss change: 0.005, Loss increases: 0]
200 [D loss: 0.684582, acc.: 50.00%] [G loss: 0.347171] [Loss change: -0.011, Loss increases: 0]
300 [D loss: 0.662876, acc.: 50.20%] [G loss: 0.345793] [Loss change: -0.003, Loss increases: 0]
400 [D loss: 0.688331, acc.: 50.20%] [G loss: 0.345598] [Loss change: -0.006, Loss increases: 0]
500 [D loss: 0.651338, acc.: 50.00%] [G loss: 0.359861] [Loss change: 0.012, Loss increases: 0]
600 [D loss: 0.663702, acc.: 50.00%] [G loss: 0.3464

1300 [D loss: 1.684733, acc.: 50.00%] [G loss: 0.056058] [Loss change: 0.006, Loss increases: 1]
1400 [D loss: 1.688379, acc.: 50.00%] [G loss: 0.048465] [Loss change: -0.003, Loss increases: 0]
1500 [D loss: 1.649527, acc.: 50.00%] [G loss: 0.057230] [Loss change: -0.001, Loss increases: 2]
1600 [D loss: 1.637195, acc.: 50.00%] [G loss: 0.061201] [Loss change: 0.003, Loss increases: 0]
1700 [D loss: 1.643798, acc.: 50.00%] [G loss: 0.059936] [Loss change: -0.004, Loss increases: 1]
1800 [D loss: 1.645630, acc.: 50.00%] [G loss: 0.067947] [Loss change: -0.006, Loss increases: 1]
1900 [D loss: 1.634091, acc.: 50.00%] [G loss: 0.083312] [Loss change: 0.015, Loss increases: 0]
2000 [D loss: 1.615365, acc.: 50.00%] [G loss: 0.075695] [Loss change: 0.005, Loss increases: 0]
2100 [D loss: 1.646853, acc.: 50.00%] [G loss: 0.074213] [Loss change: -0.002, Loss increases: 0]
2200 [D loss: 1.607213, acc.: 50.00%] [G loss: 0.075483] [Loss change: -0.006, Loss increases: 1]
2300 [D loss: 1.598731, 

0 [D loss: 0.390617, acc.: 50.00%] [G loss: 0.524029] [Loss change: 0.524, Loss increases: 0]
100 [D loss: 0.745132, acc.: 50.00%] [G loss: 0.339261] [Loss change: -0.004, Loss increases: 2]
200 [D loss: 0.732359, acc.: 50.20%] [G loss: 0.312758] [Loss change: -0.005, Loss increases: 0]
300 [D loss: 0.748618, acc.: 50.00%] [G loss: 0.310026] [Loss change: 0.007, Loss increases: 0]
400 [D loss: 0.741139, acc.: 50.20%] [G loss: 0.312845] [Loss change: 0.004, Loss increases: 0]
500 [D loss: 0.725130, acc.: 50.39%] [G loss: 0.305736] [Loss change: 0.001, Loss increases: 1]
600 [D loss: 0.694580, acc.: 50.39%] [G loss: 0.316786] [Loss change: 0.007, Loss increases: 1]
700 [D loss: 0.699071, acc.: 50.20%] [G loss: 0.321138] [Loss change: 0.009, Loss increases: 0]
800 [D loss: 0.719193, acc.: 50.20%] [G loss: 0.320886] [Loss change: 0.002, Loss increases: 1]
900 [D loss: 0.687155, acc.: 50.59%] [G loss: 0.338483] [Loss change: 0.007, Loss increases: 0]
1000 [D loss: 0.676906, acc.: 50.39%] [G

6000 [D loss: 1.286019, acc.: 49.22%] [G loss: 0.347759] [Loss change: 0.046, Loss increases: 0]
6100 [D loss: 1.286033, acc.: 48.83%] [G loss: 0.349053] [Loss change: 0.030, Loss increases: 0]
6200 [D loss: 1.346358, acc.: 48.05%] [G loss: 0.392896] [Loss change: 0.019, Loss increases: 1]
6300 [D loss: 1.218081, acc.: 49.41%] [G loss: 0.510860] [Loss change: 0.157, Loss increases: 0]
6400 [D loss: 1.181913, acc.: 49.22%] [G loss: 0.429709] [Loss change: 0.042, Loss increases: 1]
6500 [D loss: 1.208213, acc.: 49.41%] [G loss: 0.434765] [Loss change: 0.013, Loss increases: 1]
6600 [D loss: 1.170781, acc.: 49.61%] [G loss: 0.392846] [Loss change: -0.046, Loss increases: 0]
6700 [D loss: 1.068784, acc.: 50.00%] [G loss: 0.483136] [Loss change: 0.025, Loss increases: 1]
6800 [D loss: 1.119442, acc.: 49.22%] [G loss: 0.465736] [Loss change: 0.015, Loss increases: 0]
6900 [D loss: 1.092659, acc.: 49.61%] [G loss: 0.497908] [Loss change: 0.021, Loss increases: 0]
Generated Satan attacks: 
[[ 

200 [D loss: 1.557178, acc.: 50.00%] [G loss: 0.093125] [Loss change: -0.010, Loss increases: 0]
300 [D loss: 1.749864, acc.: 50.00%] [G loss: 0.088928] [Loss change: 0.011, Loss increases: 0]
400 [D loss: 2.006478, acc.: 50.00%] [G loss: 0.055091] [Loss change: -0.007, Loss increases: 0]
500 [D loss: 2.116481, acc.: 49.41%] [G loss: 0.049431] [Loss change: 0.009, Loss increases: 0]
600 [D loss: 2.134297, acc.: 49.80%] [G loss: 0.035462] [Loss change: 0.002, Loss increases: 0]
700 [D loss: 2.268686, acc.: 49.61%] [G loss: 0.028311] [Loss change: -0.003, Loss increases: 0]
800 [D loss: 2.233314, acc.: 49.41%] [G loss: 0.031139] [Loss change: -0.007, Loss increases: 1]
900 [D loss: 2.216109, acc.: 49.22%] [G loss: 0.034163] [Loss change: 0.001, Loss increases: 0]
1000 [D loss: 2.116244, acc.: 49.02%] [G loss: 0.030685] [Loss change: -0.001, Loss increases: 0]
1100 [D loss: 2.071969, acc.: 49.61%] [G loss: 0.033656] [Loss change: -0.010, Loss increases: 1]
1200 [D loss: 2.071947, acc.: 49

...

700 [D loss: 0.557574, acc.: 51.56%] [G loss: 0.442168] [Loss change: -0.003, Loss increases: 2]
800 [D loss: 0.573001, acc.: 51.17%] [G loss: 0.432354] [Loss change: -0.001, Loss increases: 1]
900 [D loss: 0.549956, acc.: 51.56%] [G loss: 0.440843] [Loss change: 0.010, Loss increases: 0]
1000 [D loss: 0.549608, acc.: 50.59%] [G loss: 0.437821] [Loss change: -0.005, Loss increases: 0]
1100 [D loss: 0.556765, acc.: 50.98%] [G loss: 0.441623] [Loss change: 0.003, Loss increases: 0]
1200 [D loss: 0.547115, acc.: 50.59%] [G loss: 0.449262] [Loss change: 0.003, Loss increases: 0]
Stoping on iteration:  1247
Generated Satan attacks: 
[[ 0.  0.  0. 14.  4.  0.  0.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  7.  0.  0.  0.  0.  0.  0. 18.  0.  0.  0.  0.  6.  9.
   0.  0. 16.  7.  0.]
 [ 0.  0.  0. 12.  3.  0.  0.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  5.  8.
   0.  0. 14.  7.  0.]]
Attack type: ipsweep.

6600 [D loss: 0.756086, acc.: 51.37%] [G loss: 0.467150] [Loss change: -0.012, Loss increases: 0]
6700 [D loss: 0.734689, acc.: 51.76%] [G loss: 0.494709] [Loss change: -0.011, Loss increases: 1]
6800 [D loss: 0.710014, acc.: 50.78%] [G loss: 0.551053] [Loss change: 0.016, Loss increases: 0]
6900 [D loss: 0.671458, acc.: 51.37%] [G loss: 0.539143] [Loss change: 0.005, Loss increases: 0]
Generated Satan attacks: 
[[  0.   0.  34.   0.   0.  32.   0.   0.   0.   0.   0.   0.   3.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   3.   0.   0.   0.   0.
    0.  27.   6.   0.   0.  63.   0.   0.   0.   0.   0.  26.   0.]
 [  0.   0.   9.   0.  37.   6.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0. 117.   0.   0.   7.  19.   0.   0.   0.   3.   0.]]
Attack type: ipsweep.     number predicted:  2
Attack type: neptune.     number predicted:  45
Attack type: normal.     number predicted:  118
Attack type: por

0 [D loss: 0.394791, acc.: 50.00%] [G loss: 0.393754] [Loss change: 0.394, Loss increases: 0]
100 [D loss: 0.924279, acc.: 50.00%] [G loss: 0.218678] [Loss change: 0.001, Loss increases: 0]
200 [D loss: 0.982786, acc.: 50.00%] [G loss: 0.196305] [Loss change: 0.001, Loss increases: 0]
300 [D loss: 0.947202, acc.: 50.00%] [G loss: 0.189964] [Loss change: -0.012, Loss increases: 1]
400 [D loss: 0.943951, acc.: 50.00%] [G loss: 0.193708] [Loss change: 0.002, Loss increases: 0]
500 [D loss: 0.932419, acc.: 50.00%] [G loss: 0.198330] [Loss change: 0.006, Loss increases: 0]
600 [D loss: 0.893307, acc.: 50.00%] [G loss: 0.201599] [Loss change: 0.007, Loss increases: 0]
700 [D loss: 0.905079, acc.: 50.00%] [G loss: 0.201467] [Loss change: 0.005, Loss increases: 0]
800 [D loss: 0.886902, acc.: 50.00%] [G loss: 0.202439] [Loss change: -0.006, Loss increases: 1]
900 [D loss: 0.867211, acc.: 50.00%] [G loss: 0.210810] [Loss change: 0.004, Loss increases: 0]
1000 [D loss: 0.845239, acc.: 50.00%] [G

0 [D loss: 0.362751, acc.: 64.06%] [G loss: 0.628469] [Loss change: 0.628, Loss increases: 0]
100 [D loss: 0.550079, acc.: 50.20%] [G loss: 0.431465] [Loss change: -0.007, Loss increases: 2]
200 [D loss: 0.594070, acc.: 50.59%] [G loss: 0.397329] [Loss change: -0.003, Loss increases: 0]


KeyboardInterrupt: 